# Reserve and configure FABRIC resources for "setting intermediate network"

## Set up your FABRIC environment

This assumes that you have already configured your FABRIC account and your Jupyter environment as described in [Hello, FABRIC](https://teaching-on-testbeds.github.io/blog/hello-fabric).

In [519]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
fablib.show_config()

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,073ee843-2310-45bd-a01f-a15d808827dc
Bastion Username,vinita_p_0000073925
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,073ee843-2310-45bd-a01f-a15d808827dc
Bastion Username,vinita_p_0000073925
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


In [520]:
!chmod 600 {fablib.get_bastion_key_filename()}
!chmod 600 {fablib.get_default_slice_private_key_file()}

## Create and submit a slice

In [580]:
slice_name="intermediate-network-2"
slice = fablib.new_slice(name=slice_name)

We will now set up all the parameters required to realize the network:

In [581]:
import random
#number of routers in access network similar irrespective of cdn or not
#redundancy_AS_1 parameter sets the hop at which redundant paths will be set
hops_AS_1=random.randint(3,6)
redundancy_AS_1=3

# For cdn, define
# number of autonomous systems including source and target AS (ranges between 2-4)
# hops seen in destination network (ranges between 3-4)
# hops seen in intermediate network (ranges bewteen 1-2)

number_of_AS_cdn=random.randint(2,4)
hops_final_AS_cdn=random.randint(3,4)
max_hops_intermediate_cdn=3

# For non CDN, define
# number of autonomous systems including source and target AS (ranges between 3-5)
# hops seen in destination network (ranges between 5-8)
# hops seen in intermediate network (ranges bewteen 1-4)

number_of_AS_not_cdn=random.randint(3,5)
hops_final_AS_not_cdn=random.randint(5,8)
max_hops_intermediate_not_cdn=4

# Set cdn=True if CDN type opf network is required else False
cdn=True
sites=[]
total_routers=[]
routers_conf=[]
routers_net=[]



In [582]:
# Print all the parameters:
print("Hops in first AS: ", hops_AS_1)
print("Number of AS: ", number_of_AS_cdn if cdn else number_of_AS_not_cdn)
print("Hops in final AS: ", hops_final_AS_cdn if cdn else hops_final_AS_not_cdn )


Hops in first AS:  3
Number of AS:  2
Hops in final AS:  4


In [583]:
# Set up the routers and networks in all the autonomous systems
if cdn:
    number_of_AS=number_of_AS_cdn
    hops_final_AS=hops_final_AS_cdn
    max_hops_intermediate=max_hops_intermediate_cdn
    
else:    
    number_of_AS=number_of_AS_not_cdn
    hops_final_AS=hops_final_AS_not_cdn
    max_hops_intermediate=max_hops_intermediate_not_cdn


# set up the number of routers in first AS

data_routers=[create_AS_3(0, routers_AS_1, redundancy_AS_1)]

  
# set up the number of routers in intermediate AS
# hops in intermediate AS is randomly selected between 1 and max_hops_intermediate value
# redundant path can start at any value between 2nd hop and hops_intermediate-1 in the intermediate AS

for i in range(1, number_of_AS-1):
    hops_intermediate=random.randint(1,max_hops_intermediate)
    print(hops_intermediate)
    data_routers+=[create_AS_3(i, hops_intermediate, r=random.randint(1,hops_intermediate))]

# set up the number of routers in Destination AS
# redundant path can start at any value between 2nd hop and hops_final_AS-1 in the destination AS
data_routers+=[create_AS_3(number_of_AS-1, hops_final_AS, random.randint(1,hops_final_AS))]

#print the routers, configuration and network data
total_routers+=[data_routers[i][0] for i in range(number_of_AS)]
routers_conf+=[data_routers[i][1] for i in range(number_of_AS)]
routers_net+=[data_routers[i][2] for i in range(number_of_AS)]
print(total_routers)
print(routers_conf)
print(routers_net)



# for i,r in enumerate(routers_net):
#     for j in range(len(r)):
#         print(len(r[j]["nodes"]))
#         print((r[j]["nodes"]))

#print(sum([len(routers_net[i][j]["nodes"]) for i,r in enumerate(routers_net) for j in range(len(r)) ]))

[[1, 1, 1, 3, 1, 1], [1, 1, 1, 1, 1]]
[[{'name': 'as0-r-0-0', 'cores': 8, 'ram': 16, 'disk': 25, 'image': 'default_ubuntu_22', 'packages': []}, {'name': 'as0-r-1-0', 'cores': 8, 'ram': 16, 'disk': 25, 'image': 'default_ubuntu_22', 'packages': []}, {'name': 'as0-r-2-0', 'cores': 8, 'ram': 16, 'disk': 25, 'image': 'default_ubuntu_22', 'packages': []}, {'name': 'as0-r-3-0', 'cores': 8, 'ram': 16, 'disk': 25, 'image': 'default_ubuntu_22', 'packages': []}, {'name': 'as0-r-3-1', 'cores': 8, 'ram': 16, 'disk': 25, 'image': 'default_ubuntu_22', 'packages': []}, {'name': 'as0-r-3-2', 'cores': 8, 'ram': 16, 'disk': 25, 'image': 'default_ubuntu_22', 'packages': []}, {'name': 'as0-r-4-0', 'cores': 8, 'ram': 16, 'disk': 25, 'image': 'default_ubuntu_22', 'packages': []}, {'name': 'as0-r-5-0', 'cores': 8, 'ram': 16, 'disk': 25, 'image': 'default_ubuntu_22', 'packages': []}], [{'name': 'as1-r-0-0', 'cores': 8, 'ram': 16, 'disk': 25, 'image': 'default_ubuntu_22', 'packages': []}, {'name': 'as1-r-1-0', 

In [577]:
# For dedicated NIC between each router pair (subnet and IP address yet to set)
def create_AS_1(as_num, n_hops, r):
    routers=[1]*min(r, n_hops) + [random.randint(1,3) for i in range(max(0, n_hops - r - 1))] + [1]
    node_conf = [ {'name': "as" + str(as_num) + '-r-' + str(j) + '-' + str(i) ,
        'cores': 8, 'ram': 16, 'disk': 25, 'image': 'default_ubuntu_22', 'packages': []}
         for j,level in enumerate(routers) for i in range(level)]
    net_conf = [
 {"name": "as"+str(as_num)+"-net"+str(j)+str(i)+"-"+str(j+1)+str(k), "subnet": "10."+str(as_num)+"."+str(j)+".0/24", 
  "nodes": [{"name": "as" + str(as_num) + '-r-' + str(j) + '-' + str(i),   "addr": "10."+str(as_num)+"."+str(j)+"."+str(j)} ]+ 
  [{"name": "as" + str(as_num) + '-r-' + str(j+1) + '-' + str(k),   "addr": "10."+str(as_num)+"."+str(j)+"."+str(j+1)}]} 
        for j,level in enumerate(routers[:-1]) for i in range(level) for k in range(routers[j+1])]
    return [routers, node_conf, net_conf]


This cell will try to identify a site that has sufficient resources for your experiment. But, you should still check in the [FABRIC Portal](https://portal.fabric-testbed.net/resources/all) to make sure it is not in maintenance or out of service - if so, you should run the cell again until you get a site that is ready to use.

In [575]:
# For LAN between a router in i-th hop and all routers in (i+1)th hop


def create_AS_2(as_num, n_hops, r):
    routers=[1]*min(r, n_hops) + [random.randint(1,3) for i in range(max(0, n_hops - r - 1))] + [1]
    node_conf = [ {'name': "as" + str(as_num) + '-r-' + str(j) + '-' + str(i) ,
        'cores': 8, 'ram': 16, 'disk': 25, 'image': 'default_ubuntu_22', 'packages': []}
         for j,level in enumerate(routers) for i in range(level)]
    net_conf = [
 {"name": "as"+str(as_num)+"-net"+str(j)+str(i)+"-"+str(j+1), "subnet": "10."+str(as_num)+"."+str(j)+str(i)+".0/24", 
  "nodes": [{"name": "as" + str(as_num) + '-r-' + str(j) + '-' + str(i),   "addr": "10."+str(as_num)+"."+str(j)+str(i)+"."+str(1)}]+ 
  [{"name": "as" + str(as_num) + '-r-' + str(j+1) + '-' + str(k),   "addr": "10."+str(as_num)+"."+str(j)+str(i)+"."+str(k+2)}for k in range(routers[j+1])]} 
        for j,level in enumerate(routers[:-1]) for i in range(level)]

    return [routers, node_conf, net_conf]


In [576]:
# For LAN between all routers in i-th hop and all routers in (i+1)th hop
def create_AS_3(as_num, n_hops, r):
    routers=[1]*min(r, n_hops) + [random.randint(1,3) for i in range(max(0, n_hops - r - 1))] + [1]
    node_conf = [ {'name': "as" + str(as_num) + '-r-' + str(j) + '-' + str(i) ,
        'cores': 8, 'ram': 16, 'disk': 25, 'image': 'default_ubuntu_22', 'packages': []}
         for j,level in enumerate(routers) for i in range(level)]
    net_conf = [
 {"name": "as"+str(as_num)+"-net"+str(j)+str(j+1), "subnet": "10."+str(as_num)+"."+str(j)+".0/24", 
  "nodes": [{"name": "as" + str(as_num) + '-r-' + str(j) + '-' + str(i),   "addr": "10."+str(as_num)+"."+str(j)+"."+str(i+1)} for i in range(level)]+ 
  [{"name": "as" + str(as_num) + '-r-' + str(j+1) + '-' + str(i),   "addr": "10."+str(as_num)+"."+str(j)+"."+str(level+i+1)}for i in range(routers[j+1])]} 
        for j,level in enumerate(routers[:-1])]
    return [routers, node_conf, net_conf]

In [584]:
# Get the sites
for i,r in enumerate(total_routers):   
    exp_requires = {'core': (sum([(routers_conf[i][j]["cores"]) for j in range(len(routers_conf[i])) ])), 
                    'smart_nic_1': (sum([len(routers_net[i][j]["nodes"]) for j in range(len(routers_net[i])) ]))}
    print(exp_requires)
    while True:
        site_name = fablib.get_random_site(avoid=sites)
        if ( (fablib.resources.get_core_available(site_name) > 1.2*exp_requires['core']) and
            (fablib.resources.get_component_available(site_name, 'SharedNIC-ConnectX-6') > 1.2**exp_requires['smart_nic_1']) ):
            break

    fablib.show_site(site_name)
    sites.append(site_name)
    

{'core': 64, 'smart_nic_1': 14}


Name,NCSA
State,Active
Address,"1725 S Oak St.,Champaign, IL 61820"
Location,"(40.1035624, -88.2415105)"
Hosts,3
CPUs,6
Cores Available,338
Cores Capacity,384
Cores Allocated,46
RAM Available,1370
RAM Capacity,1506


{'core': 40, 'smart_nic_1': 8}


Name,PSC
State,Active
Address,"4350 Northern Pike,Monroeville, PA 15146"
Location,"(40.43437047818922, -79.75038781539575)"
Hosts,3
CPUs,6
Cores Available,326
Cores Capacity,384
Cores Allocated,58
RAM Available,602
RAM Capacity,1506


In [568]:
# file = open(slice_name+".txt", "w")
# file.write(str(total_routers))
# file.close()

In [585]:
print(sites)

['', 'NCSA', 'PSC']


In [586]:
for i,r in enumerate(routers_conf):
     for j in range(len(r)):
        print(r[j]['name'],sites[i+1], r[j]['cores'], r[j]['ram'], r[j]['disk'], r[j]['image'])

as0-r-0-0 NCSA 8 16 25 default_ubuntu_22
as0-r-1-0 NCSA 8 16 25 default_ubuntu_22
as0-r-2-0 NCSA 8 16 25 default_ubuntu_22
as0-r-3-0 NCSA 8 16 25 default_ubuntu_22
as0-r-3-1 NCSA 8 16 25 default_ubuntu_22
as0-r-3-2 NCSA 8 16 25 default_ubuntu_22
as0-r-4-0 NCSA 8 16 25 default_ubuntu_22
as0-r-5-0 NCSA 8 16 25 default_ubuntu_22
as1-r-0-0 PSC 8 16 25 default_ubuntu_22
as1-r-1-0 PSC 8 16 25 default_ubuntu_22
as1-r-2-0 PSC 8 16 25 default_ubuntu_22
as1-r-3-0 PSC 8 16 25 default_ubuntu_22
as1-r-4-0 PSC 8 16 25 default_ubuntu_22


In [587]:
for i,r in enumerate(routers_conf):
     for j in range(len(r)):
        slice.add_node(name=r[j]['name'], site=sites[i+1], 
                       cores=r[j]['cores'], 
                       ram=r[j]['ram'], 
                       disk=r[j]['disk'], 
                       image=r[j]['image'])


In [588]:
print(slice.list_nodes())

ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
None,as0-r-0-0,None,None,None,default_ubuntu_22,qcow2,None,NCSA,ubuntu,,None,,,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
None,as0-r-1-0,None,None,None,default_ubuntu_22,qcow2,None,NCSA,ubuntu,,None,,,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
None,as0-r-2-0,None,None,None,default_ubuntu_22,qcow2,None,NCSA,ubuntu,,None,,,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
None,as0-r-3-0,None,None,None,default_ubuntu_22,qcow2,None,NCSA,ubuntu,,None,,,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
None,as0-r-3-1,None,None,None,default_ubuntu_22,qcow2,None,NCSA,ubuntu,,None,,,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
None,as0-r-3-2,None,None,None,default_ubuntu_22,qcow2,None,NCSA,ubuntu,,None,,,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
None,as0-r-4-0,None,None,None,default_ubuntu_22,qcow2,None,NCSA,ubuntu,,None,,,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
None,as0-r-5-0,None,None,None,default_ubuntu_22,qcow2,None,NCSA,ubuntu,,None,,,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
None,as1-r-0-0,None,None,None,default_ubuntu_22,qcow2,None,PSC,ubuntu,,None,,,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
None,as1-r-1-0,None,None,None,default_ubuntu_22,qcow2,None,PSC,ubuntu,,None,,,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


In [ ]:
# for i,r in enumerate(routers_net):
#     print(len(r))
#     print(r)
#     for j in range(len(r)):
#         for node in r[j]['nodes']:
#             print(node["name"])
#         print(r[j])

In [589]:
# this cell sets up the network links 
for i,r in enumerate(routers_net):
     for j in range(len(r)):
        ifaces = [slice.get_node(node["name"]).add_component(model="NIC_Basic", 
                                                     name=r[j]["name"]).get_interfaces()[0] for node in r[j]['nodes'] ]
        slice.add_l2network(name=r[j]["name"], type='L2Bridge', interfaces=ifaces)


When everything is ready, submit the slice and wait for it to get to "StableOK" state.  You can also see the state of your slice on the browser-based interface at https://portal.fabric-testbed.net/experiments#slices.

In case of an error, you can modify the slice name in the first cell of the "Create and submit a slice" section, to try again with a new slice (different name). Then, return to this cell and click Run > Run All Above Selected Cell. Finally, re-run the `slice.submit()` cell.

In [590]:
slice.submit()


Retry: 13, Time: 1242 sec


ID,9909bad5-3f1c-40a9-94a7-7f058e12dabb
Name,intermediate-network-2
Lease Expiration (UTC),2023-07-26 08:15:15 +0000
Lease Start (UTC),2023-07-25 08:15:16 +0000
Project ID,073ee843-2310-45bd-a01f-a15d808827dc
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
6bd71b30-6044-40ef-b34b-b0b936aee58f,as0-r-0-0,8,16,100,default_ubuntu_22,qcow2,ncsa-w2.fabric-testbed.net,NCSA,ubuntu,2620:0:c80:1001:f816:3eff:fe51:7c7f,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1001:f816:3eff:fe51:7c7f,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
d065f78d-1be9-49de-b5df-c24e1463c70b,as0-r-1-0,8,16,100,default_ubuntu_22,qcow2,ncsa-w2.fabric-testbed.net,NCSA,ubuntu,2620:0:c80:1001:f816:3eff:fe5d:ee04,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1001:f816:3eff:fe5d:ee04,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
850b009c-ea50-45bf-a7da-628b5e0bbd56,as0-r-2-0,8,16,100,default_ubuntu_22,qcow2,ncsa-w2.fabric-testbed.net,NCSA,ubuntu,2620:0:c80:1001:f816:3eff:fe71:c4a7,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1001:f816:3eff:fe71:c4a7,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
22eaa215-2f80-43bf-8ca0-a1b2b372f7b4,as0-r-3-0,8,16,100,default_ubuntu_22,qcow2,ncsa-w2.fabric-testbed.net,NCSA,ubuntu,2620:0:c80:1001:f816:3eff:fe74:fc07,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1001:f816:3eff:fe74:fc07,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
4bbabfcf-355e-47e2-b683-fecb0c74c451,as0-r-3-1,8,16,100,default_ubuntu_22,qcow2,ncsa-w2.fabric-testbed.net,NCSA,ubuntu,2620:0:c80:1001:f816:3eff:fe3e:bb87,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1001:f816:3eff:fe3e:bb87,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
2d1085b3-6ea4-4f2d-8854-406fdd4ca0be,as0-r-3-2,8,16,100,default_ubuntu_22,qcow2,ncsa-w2.fabric-testbed.net,NCSA,ubuntu,2620:0:c80:1001:f816:3eff:fe25:c4ab,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1001:f816:3eff:fe25:c4ab,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
48a35c25-a658-4286-8efc-c064ef1c16d0,as0-r-4-0,8,16,100,default_ubuntu_22,qcow2,ncsa-w2.fabric-testbed.net,NCSA,ubuntu,2620:0:c80:1001:f816:3eff:fe5a:940e,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1001:f816:3eff:fe5a:940e,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
7ae6934b-896b-4d32-acd8-66c056e4092a,as0-r-5-0,8,16,100,default_ubuntu_22,qcow2,ncsa-w2.fabric-testbed.net,NCSA,ubuntu,2620:0:c80:1001:f816:3eff:fe24:7913,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1001:f816:3eff:fe24:7913,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
5126ebc1-c09c-4aae-9d8b-1420fdb2598c,as1-r-0-0,8,16,100,default_ubuntu_22,qcow2,psc-w1.fabric-testbed.net,PSC,ubuntu,2001:5e8:ff00:ffff:f816:3eff:fedb:7fac,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:5e8:ff00:ffff:f816:3eff:fedb:7fac,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
6dadbda9-b1ee-4ec4-aa8c-3252e3825459,as1-r-1-0,8,16,100,default_ubuntu_22,qcow2,psc-w1.fabric-testbed.net,PSC,ubuntu,2001:5e8:ff00:ffff:f816:3eff:fe09:9159,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:5e8:ff00:ffff:f816:3eff:fe09:9159,/home/fabric/work/fabric_config/slice_key.pub,/home/

ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
8efe109d-83e6-4caa-acd6-5c44cb6c74d0,as0-net01,L2,L2Bridge,NCSA,None,None,Active,
0a0104f4-ec21-47ca-8c12-b02c02a08e3b,as0-net12,L2,L2Bridge,NCSA,None,None,Active,
225770bd-fe08-4c14-b489-e6a3bf1ba289,as0-net23,L2,L2Bridge,NCSA,None,None,Active,
0305e8a0-94e6-4f1a-a3cc-b6be98528c21,as0-net34,L2,L2Bridge,NCSA,None,None,Active,
1afb8fc1-9802-4358-8368-81e0700148d5,as0-net45,L2,L2Bridge,NCSA,None,None,Active,
c5c505f8-d99e-4dcf-a0c4-205e938766da,as1-net01,L2,L2Bridge,PSC,None,None,Active,
6c2fd8f4-4a17-456e-a1ff-2794dfa374f6,as1-net12,L2,L2Bridge,PSC,None,None,Active,
02302d80-6910-4760-a302-e686726db19f,as1-net23,L2,L2Bridge,PSC,None,None,Active,
a1bb86c0-48db-4e57-99d7-dbfa0ff81b6b,as1-net34,L2,L2Bridge,PSC,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node
as0-r-0-0-as0-net01-p1,p1,as0-r-0-0,as0-net01,100,config,,12:53:E3:FD:D0:2E,ens7,ens7,None,4
as0-r-1-0-as0-net01-p1,p1,as0-r-1-0,as0-net01,100,config,,1A:06:19:1F:2B:08,ens8,ens8,None,4
as0-r-1-0-as0-net12-p1,p1,as0-r-1-0,as0-net12,100,config,,16:E2:21:CF:C2:97,ens7,ens7,None,4
as0-r-2-0-as0-net12-p1,p1,as0-r-2-0,as0-net12,100,config,,12:9B:F7:5D:64:BC,ens7,ens7,None,4
as0-r-2-0-as0-net23-p1,p1,as0-r-2-0,as0-net23,100,config,,12:A6:83:DE:92:B0,ens8,ens8,None,4
as0-r-3-0-as0-net23-p1,p1,as0-r-3-0,as0-net23,100,config,,1A:34:82:40:E8:76,ens8,ens8,None,4
as0-r-3-0-as0-net34-p1,p1,as0-r-3-0,as0-net34,100,config,,1A:19:E3:1B:1D:08,ens7,ens7,None,4
as0-r-3-1-as0-net34-p1,p1,as0-r-3-1,as0-net34,100,config,,12:B2:D2:8A:4B:71,ens7,ens7,None,4
as0-r-3-1-as0-net23-p1,p1,as0-r-3-1,as0-net23,100,config,,12:FC:58:E0:23:5C,ens8,ens8,None,4
as0-r-3-2-as0-net23-p1,p1,as0-r-3-2,as0-net23,100,config,,1A:9E:9B:43:74:D7,ens8,ens8,None,4



Time to print interfaces 1272 seconds


'9909bad5-3f1c-40a9-94a7-7f058e12dabb'

## Configure resources

In [591]:
slice.get_state()

'StableOK'

In [592]:
slice.wait_ssh(progress=True)

Waiting for slice . Slice state: StableOK
Waiting for ssh in slice . ssh successful


True

Bring up all of the network interfaces:

In [ ]:
# for iface in slice.get_interfaces():
#     iface.ip_link_up()

In [ ]:
# for i,r in enumerate(routers_net):
#      for j in range(len(r)):
#         for node in r[j]['nodes']:
#             if_name = node['name'] + '-' + r[j]['name'] + '-p1'
#             print(if_name)
#             print("addr=",node['addr'], "subnet=",r[j]['subnet'])

In [594]:
from ipaddress import ip_address, IPv4Address, IPv4Network

for i,r in enumerate(routers_net):
     for j in range(len(r)):
        for node in r[j]['nodes']:
            if_name = node['name'] + '-' + r[j]['name'] + '-p1'
            iface = slice.get_interface(if_name)
            iface.ip_link_up()
            iface.ip_addr_add(addr=node['addr'], subnet=IPv4Network(r[j]['subnet']))



In [595]:
router_nodes=[]
for i,r in enumerate(routers_conf):
    nodes=[slice.get_node(name=r[j]['name']) for j in range(len(r))]
    router_nodes.append(nodes)
    

In [516]:
# router_nodes=[]
# for i,r in enumerate(routers_conf):
#     nodes=[r[j]['name'] for j in range(len(r))]
#     router_nodes.append(nodes)

In [ ]:
# print(len(router_nodes))

In [ ]:
# print(router_nodes)

In [596]:
for i,nodes in enumerate(router_nodes):
    for n in nodes:
        print(n.get_name())
        n.execute("sudo sysctl -w net.ipv4.ip_forward=1")
        n.execute("sudo apt-get update; sudo apt-get -y install net-tools", quiet=True)
        n.execute("wget -O - https://git.io/JYhs5 | bash")
    

as0-r-0-0
net.ipv4.ip_forward = 1
 --2023-07-25 08:44:21--  https://git.io/JYhs5
Resolving git.io (git.io)... 140.82.113.22
Connecting to git.io (git.io)|140.82.113.22|:443... failed: Network is unreachable.
 as0-r-1-0
net.ipv4.ip_forward = 1
 --2023-07-25 08:44:45--  https://git.io/JYhs5
Resolving git.io (git.io)... 140.82.113.22
Connecting to git.io (git.io)|140.82.113.22|:443... failed: Network is unreachable.
 as0-r-2-0
net.ipv4.ip_forward = 1
 --2023-07-25 08:45:08--  https://git.io/JYhs5
Resolving git.io (git.io)... 140.82.112.21
Connecting to git.io (git.io)|140.82.112.21|:443... failed: Network is unreachable.
 as0-r-3-0
net.ipv4.ip_forward = 1
 --2023-07-25 08:45:31--  https://git.io/JYhs5
Resolving git.io (git.io)... 140.82.112.22
Connecting to git.io (git.io)|140.82.112.22|:443... failed: Network is unreachable.
 as0-r-3-1
net.ipv4.ip_forward = 1
 --2023-07-25 08:45:53--  https://git.io/JYhs5
Resolving git.io (git.io)... 140.82.112.22
Connecting to git.io (git.io)|140.82.112

In [597]:
for x,nodes in enumerate(router_nodes):
    for n in nodes:
        print(n)
        n.execute("ifconfig -a")

-----------------  -----------------------------------------------------------------------------------------------------------------------------------------
ID                 6bd71b30-6044-40ef-b34b-b0b936aee58f
Name               as0-r-0-0
Cores              8
RAM                16
Disk               100
Image              default_ubuntu_22
Image Type         qcow2
Host               ncsa-w2.fabric-testbed.net
Site               NCSA
Management IP      2620:0:c80:1001:f816:3eff:fe51:7c7f
Reservation State  Active
Error Message
SSH Command        ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1001:f816:3eff:fe51:7c7f
-----------------  -----------------------------------------------------------------------------------------------------------------------------------------
ens3: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 9000
        inet 10.30.6.116  netmask 255.255.254.0  broadcast 10.30.7.255
        inet6 fe80::f

In [ ]:
for x,nodes in enumerate(router_nodes):
    for n in nodes:
        n.execute("VTYSH_PAGER=more; sudo vtysh; sudo vtysh -E -c'configure terminal\nrouter ospf\nnetwork 10.10.0.0/16 area 0.0.0.0\n exit\n exit\n exit'  ")

In [ ]:
for x,nodes in enumerate(router_nodes):
    for n in nodes:
        n.execute("VTYSH_PAGER=more; sudo vtysh; sudo vtysh -E -c 'show ip route\nexit'")

In [ ]:
# slice.delete()

In [598]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

# Set end date to 3 days from now
end_date = (datetime.now(timezone.utc) + timedelta(days=4)).strftime("%Y-%m-%d %H:%M:%S %z")
slice.renew(end_date)